In [21]:
!nvidia-smi

Thu Mar 30 09:53:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   30C    P0    44W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
# sync python module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Gen pretrain data dict json

In [28]:
workspace_dir = '/nfs/Workspace/CardiacSeg'
import sys
import importlib

# set package path
sys.path.append(workspace_dir)

from data_utils.io import save_json
from expers.preprocess import get_rel_data_dicts

root_data_dir = os.path.join(workspace_dir, 'dataset')
data_name = 'image_cas'
exp_name = 'exp_50'
data_dir = os.path.join(root_data_dir, data_name)
data_dicts_json = os.path.join(workspace_dir, 'exps', 'data_dicts', data_name, f'{exp_name}.json')


dataset = importlib.import_module(f'datasets.{data_name}_dataset')
get_data_dicts = getattr(dataset, 'get_data_dicts', None)

data_dicts = get_rel_data_dicts(get_data_dicts(data_dir))

# data_dicts
tr_data_dicts = data_dicts[:-10]
val_data_dicts = data_dicts[-10:]
tt_data_dicts = data_dicts[-10:]

out_data_dict = {
    'train':tr_data_dicts,
    'val':val_data_dicts,
    'test':val_data_dicts
}

save_json(out_data_dict, data_dicts_json)
out_data_dict

save json to /nfs/Workspace/CardiacSeg/exps/data_dicts/image_cas/exp_50.json


{'train': [{'image': '10016975/img.nii.gz', 'label': '10016975/label.nii.gz'},
  {'image': '10017784/img.nii.gz', 'label': '10017784/label.nii.gz'},
  {'image': '10018985/img.nii.gz', 'label': '10018985/label.nii.gz'},
  {'image': '10027729/img.nii.gz', 'label': '10027729/label.nii.gz'},
  {'image': '10029129/img.nii.gz', 'label': '10029129/label.nii.gz'},
  {'image': '10032743/img.nii.gz', 'label': '10032743/label.nii.gz'},
  {'image': '10033946/img.nii.gz', 'label': '10033946/label.nii.gz'},
  {'image': '10039455/img.nii.gz', 'label': '10039455/label.nii.gz'},
  {'image': '10043087/img.nii.gz', 'label': '10043087/label.nii.gz'},
  {'image': '10048510/img.nii.gz', 'label': '10048510/label.nii.gz'},
  {'image': '10050634/img.nii.gz', 'label': '10050634/label.nii.gz'},
  {'image': '10054193/img.nii.gz', 'label': '10054193/label.nii.gz'},
  {'image': '10054710/img.nii.gz', 'label': '10054710/label.nii.gz'},
  {'image': '10055172/img.nii.gz', 'label': '10055172/label.nii.gz'},
  {'image':

## Pretrain

In [35]:
import os

workspace_dir = '/nfs/Workspace/CardiacSeg'

exp_name = 'exp_50'
model_name = 'swinunetr'
data_name = 'image_cas'

root_data_dir = os.path.join(workspace_dir, 'dataset')
data_dir = os.path.join(root_data_dir, data_name)
data_dicts_json = os.path.join(workspace_dir, 'exps', 'data_dicts', data_name, f'{exp_name}.json')

root_dir = os.path.join(
    workspace_dir,
    'exps',
    'exps',
    model_name,
    data_name,
    'pretrain',
    exp_name
)
model_dir = os.path.join(root_dir, 'models')
log_dir = os.path.join(root_dir, 'logs')

os.makedirs(root_dir, exist_ok=True)

best_model_path = os.path.join(model_dir, 'model_bestValRMSE.pt')
final_model_path = os.path.join(model_dir, 'model_final.pt')

%cd {root_dir}

/nfs/Workspace/CardiacSeg/exps/exps/swinunetr/image_cas/pretrain/exp_50


In [37]:
# note patch_size effect shape
!PYTHONPATH={workspace_dir} /opt/conda/bin/python {workspace_dir}/expers/pretrain/train.py\
--use_checkpoint \
--patch_size=2 \
--batch_size=1 \
--num_steps=100000 \
--lrdecay \
--eval_num=2 \
--max_early_stop_count=60 \
--model_name={model_name} \
--logdir={log_dir} \
--model_dir={model_dir} \
--data_dirs={data_dir} \
--data_dicts_jsons={data_dicts_json} \
# --resume={final_model_path}
# --lr=<Lr> 
# --roi_x=<Roi_x> \
# --roi_y=<Roi_y> \
# --roi_z=<Roi_z> \

cuda is available
Training with a single process on 1 GPUs.
model: swinunetr
use warmup cosine schedule
load data dict jsons: ['/nfs/Workspace/CardiacSeg/exps/data_dicts/image_cas/exp_50.json']
load json from /nfs/Workspace/CardiacSeg/exps/data_dicts/image_cas/exp_50.json
train files (40): ['img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img']
val files (10): ['img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img']
test files (10): ['img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img', 'img']
Dataset 1 ['/nfs/Workspace/CardiacSeg/dataset/image_cas']: number of tr data: 40
Dataset 1 ['/nfs/Workspace/CardiacSeg/dataset/image_cas']: number of val data: 10
Dataset all training: number of data: 40
Dataset all validation: number of data: 10
<class

In [26]:
import sys
# set package path
sys.path.append(workspace_dir)

from networks.network import network
from networks.ssl_head import SSLHead

import torch

class args:
    in_channels=1
    out_channels=2
    device='cuda'

net = network(model_name, args)

model = SSLHead(
  args.in_channels,
  net, 
  dim=768
).to(args.device)

model_pth = best_model_path
model_dict = torch.load(model_pth)
model.load_state_dict(model_dict["state_dict"])

if "global_step" in model_dict:
    global_step = model_dict["global_step"]
if "early_stop_count" in model_dict:
    early_stop_count = model_dict["early_stop_count"]
if "val_best" in model_dict:
    best_val = model_dict["val_best"]
    
print('load model:', model_pth)
print('global step:', global_step)
print('best val:', best_val)

model: unetcnx_x3_2_2
load model: /nfs/Workspace/CardiacSeg/exps/exps/unetcnx_x3_2_2/chgh/pretrain/exp_1/models/pretrain/exp_1/model_bestValRMSE.pt
global step: 7100
best val: 0.0615180271367232
